# Creating the Channel Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.


## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    The bitcoin core functional test framework directory.
    The directory containing taproot-lightning-channels-workshop.

You'll need to edit these next two lines for your local setup.


In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"


### Setup bitcoin core test framework

Start up regtest mode, delete any regtest network history so we are starting from scratch.

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *
from functions.bip_0340_reference import *
from functions.test_framework.musig import generate_musig_key
from functions.test_framework.key import generate_bip340_key_pair 
from functions.test_framework.script import tagged_hash
from functions.test_framework.address import program_to_witness


import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-08-28T23:52:46.495000Z TestFramework (INFO): PRNG seed is: 7458706229051476539
2025-08-28T23:52:46.497000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_j48l7b72


### Setup Alice and Bob Funding Outputs

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [3]:
# Create a new Alice key pair for channel funding multisig output
alice_funding_privkey, alice_funding_pubkey = generate_bip340_key_pair()
alice_funding_pubkey_b = alice_funding_pubkey.get_bytes()
alice_funding_privkey_b = alice_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_funding_address = program_to_witness(0x01, alice_funding_pubkey_b)
print(f"Alice funding pubkey: {alice_funding_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_funding_privkey_b.hex()}")
print(f"Alice funding address: {alice_funding_address}")

# Create a new Bob key pair for channel funding multisig output
bob_funding_privkey, bob_funding_pubkey = generate_bip340_key_pair()
bob_funding_pubkey_b = bob_funding_pubkey.get_bytes()
bob_funding_privkey_b = bob_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
bob_funding_address = program_to_witness(0x01, bob_funding_pubkey_b)
print(f"Bob funding pubkey: {bob_funding_pubkey_b.hex()}")
print(f"Bob funding privkey: {bob_funding_privkey_b.hex()}")
print(f"Bob funding address: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
alice_initial_fund = 1
bob_initial_fund = 1
txid_to_spend = node.sendmany("", {alice_funding_address: alice_initial_fund, bob_funding_address: bob_initial_fund})
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding pubkey: 2d00deeee7a0c4bcf051e7f1937f3078fffe61180dacd09b9ef18bdeccd853a3
Alice funding privkey: 5ed4aeb573de3661558c77b1c65b3452589256c2b30d446902af9441412420d0
Alice funding address: bcrt1p95qdamh85rzteuz3ulcexles0rllucgcpkkdpxu77x9aanxc2w3syanpdj
Bob funding pubkey: 4bb8265005a54bd177b3660305526fce90a2e4b23349dbe303c81667fb66ba5a
Bob funding privkey: b17d515ee8e65b6ce48875dbb3f8b43da606b528bfd65c84bd578010034c3b8f
Bob funding address: bcrt1pfwuzv5q9549azaanvcps25n0e6g29e9jxdyahccreqtx07mxhfdqfacj08


## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs:

#### The change output back to Alice

In [4]:

# Create a new Alice key pair output for the change
alice_change_privkey, alice_change_pubkey = generate_bip340_key_pair()
alice_change_pubkey_b = alice_change_pubkey.get_bytes()
alice_change_privkey_b = alice_change_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_change_address = program_to_witness(0x01, alice_change_pubkey_b)
print(f"Alice funding pubkey: {alice_change_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_change_privkey_b.hex()}")
print(f"Alice funding address: {alice_change_address}")


Alice funding pubkey: 94f6258d64ceb5fc97b6d331a5ceebfdf955bb6f66953f7a820379b76cc87012
Alice funding privkey: bcb54a403f41fec96bbcd011c5d0870cdf13fdeff7d4db6c1a64d6944f0b406a
Alice funding address: bcrt1pjnmztrtye66le9ak6vc6tnhtlhu4twm0v62n775zqdumwmxgwqfq3quzc2


#### The channel musig funding output

The diagram below illustrates how a Taproot channel funding output is constructed.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| Pagg + T |
                       +----------+
                         ^      ^
                         |      |  +-----------+
                         |       --| T = t * G |<---------
                         |         +-----------+          |
                         |                                |
           +---------------------------------------+    +---+   +-------------------------------+
           |      generate_musig_key(Pa, Pb)       |    | t | = | TaggedHash ("Taptweak", Pagg) |
           +---------------------------------------+    +---+   +-------------------------------+

In [5]:
# Create a new Alice key pair for channel funding multisig output
privkey_alice_musig, pubkey_alice_musig = generate_bip340_key_pair()
print(f"Alice MuSig pubkey: {pubkey_alice_musig.get_bytes().hex()}")
print(f"Alice MuSig privkey: {privkey_alice_musig.get_bytes().hex()}")

# Create a new Bob key pair for channel funding multisig output
privkey_bob_musig, pubkey_bob_musig = generate_bip340_key_pair()
print(f"Bob MuSig pubkey: {pubkey_bob_musig.get_bytes().hex()}")
print(f"Bob MuSig privkey: {privkey_bob_musig.get_bytes().hex()}")

# Generate a 2-of-2 aggregate MuSig key using the pubkeys form Alice and Bob
c_map, agg_pubkey = generate_musig_key([pubkey_alice_musig, pubkey_bob_musig])

# Multiply individual keys with challenges
privalice_c = c_map[pubkey_alice_musig] * privkey_alice_musig
privbob_c = c_map[pubkey_bob_musig] * privkey_bob_musig
pubalice_c = c_map[pubkey_alice_musig] * pubkey_alice_musig
pubbob_c = c_map[pubkey_bob_musig] * pubkey_bob_musig

if agg_pubkey.get_y()%2 != 0:
    privalice_c.negate()
    privbob_c.negate()
    pubalice_c.negate()
    pubbob_c.negate()
    agg_pubkey.negate()

print(f"Aggregate musig pubkey: {agg_pubkey.get_bytes().hex()}")

# Tweak musig public key
# Method: ECPubKey.tweak_add()
tweak = tagged_hash("TapTweak", agg_pubkey.get_bytes())
agg_pubkey_tweaked = agg_pubkey.tweak_add(tweak)
print(f"Tweaked aggregate musig pubkey: {agg_pubkey_tweaked.get_bytes().hex()}")

agg_pubkey_tweaked_b = agg_pubkey_tweaked.get_bytes()

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
channel_funding_musig_address = program_to_witness(0x01, agg_pubkey_tweaked_b)
print(f"Channel funding musig output address: {channel_funding_musig_address}")

Alice MuSig pubkey: 170129e4125b86012530b9ec61000dffc6d65a553544cffe64e8c897d4eacb4e
Alice MuSig privkey: 3aeaf265826309bb6ce57a3eecae3a4f4eb2eac6b4f98cdadc40567a361060a8
Bob MuSig pubkey: 11b671bacfabdbbc3541a17becc8d019943b53cdbbea6c87208090ccee2d7f06
Bob MuSig privkey: 4e1cb06e225b56dd57174708145410779fc3849d03c2da4facd8105a59453238
Aggregate musig pubkey: da6156497969d08d2cf104fc36d89c3222be4860e3f363d343b5c6599342b9f8
Tweaked aggregate musig pubkey: 27137edd8ee18fbe86931445909f383e713b37c9b6ec49373c4da06743d4e79c
Channel funding musig output address: bcrt1pyufhahvwux8map5nz3zep8ec8ecnkd7fkmkyjdeufksxws75u7wq2l56fq


#### The channel funding transaction

##### The unsigned transaction

In [6]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = bytes.fromhex("00000000")

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.0001") * 100000000)

# OUTPUT 1 - Channel Funding Output
# 0.01 BTC to the 2-of-2 multisig Taproot output
output1_value_sat = int(float("0.01") * 100000000)
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output1_spk = bytes.fromhex("51") + varint_len(agg_pubkey_tweaked_b) + agg_pubkey_tweaked_b

# OUTPUT 2 - Change Output
# The rest back to Alice's change address
# 1 BTC - 0.01 BTC - 0.0001 BTC = 0.9899 BTC
output2_value_sat = int(float(alice_initial_fund) * 100000000) - output1_value_sat - tx_fee_sat
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output2_spk = bytes.fromhex("51") + varint_len(alice_change_pubkey_b) + alice_change_pubkey_b

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  02000000011c8800e19c116a74ab8eea9fb1d2d3c619f54827181ab04e6f2ae29a745b5d850000000000ffffffff0240420f000000000022512027137edd8ee18fbe86931445909f383e713b37c9b6ec49373c4da06743d4e79cb077e6050000000022512094f6258d64ceb5fc97b6d331a5ceebfdf955bb6f66953f7a820379b76cc8701200000000
{
  "txid": "1fe9373950892375539b05ccc34cf43628d9bcfb3b709665299ca7713a6b39a8",
  "hash": "1fe9373950892375539b05ccc34cf43628d9bcfb3b709665299ca7713a6b39a8",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 0,
  "vin": [
    {
      "txid": "855d5b749ae22a6f4eb01a182748f519c6d3d2b19fea8eab746a119ce100881c",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.01000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 27137edd8ee18fbe86931445909f383e713b37c9b6ec49373c4da06743d4e79c",
        "desc": "rawtr(27137edd8ee18fbe86931445909f383e713b37c9b6ec49373c4da067

##### The sighash for key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [7]:
# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")
index_of_this_input = bytes.fromhex("0000 0000")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(alice_initial_fund * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
spk = bytes.fromhex("51") + varint_len(alice_funding_pubkey_b) + alice_funding_pubkey_b

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) ######

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)


##### Signing the sighash

In [8]:
# Sign the transaction
aux_rand = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")
signature = tr.schnorr_sign(sighash, alice_funding_privkey_b, aux_rand)
# Sighash flag is not appended for SIGHASH_DEFAULT

##### The signed transaction

Finally the last step is to add the signature to the witness field and she has the signed transaction!

In [9]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(signature)
    + signature
)
print(witness.hex())

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

01409134914a869acabca5f6650098157f90da0c96bb6e93811553d3aa586b0372103c8d4d462444878a2c4f98723eef930cd6fb4eb83529907ff9f56931c8eb2e1d
signed tx:  020000000001011c8800e19c116a74ab8eea9fb1d2d3c619f54827181ab04e6f2ae29a745b5d850000000000ffffffff0240420f000000000022512027137edd8ee18fbe86931445909f383e713b37c9b6ec49373c4da06743d4e79cb077e6050000000022512094f6258d64ceb5fc97b6d331a5ceebfdf955bb6f66953f7a820379b76cc8701201409134914a869acabca5f6650098157f90da0c96bb6e93811553d3aa586b0372103c8d4d462444878a2c4f98723eef930cd6fb4eb83529907ff9f56931c8eb2e1d00000000
{
  "txid": "1fe9373950892375539b05ccc34cf43628d9bcfb3b709665299ca7713a6b39a8",
  "hash": "492def7d7c98f3a4461417663bc8a2ed0fd22b009cf0cbb4e13a7ab30bc7a309",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 0,
  "vin": [
    {
      "txid": "855d5b749ae22a6f4eb01a182748f519c6d3d2b19fea8eab746a119ce100881c",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [